# Retail Project

### Content

You are provided with historical sales data for 45 stores located in different regions - each store contains a number of departments. The company also runs several promotional markdown events throughout the year. These markdowns precede prominent holidays, the four largest of which are the Super Bowl, Labor Day, Thanksgiving, and Christmas. The weeks including these holidays are weighted five times higher in the evaluation than non-holiday weeks.

### Stores

Anonymized information about the 45 stores, indicating the type and size of store

### Features

Contains additional data related to the store, department, and regional activity for the given dates.

Store - the store number
Date - the week
Temperature - average temperature in the region
Fuel_Price - cost of fuel in the region
MarkDown1-5 - anonymized data related to promotional markdowns. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA
CPI - the consumer price index
Unemployment - the unemployment rate
IsHoliday - whether the week is a special holiday week
Sales

Historical sales data, which covers to 2010-02-05 to 2012-11-01. Within this tab you will find the following fields:

Store - the store number
Dept - the department number
Date - the week
Weekly_Sales -  sales for the given department in the given store
IsHoliday - whether the week is a special holiday week
The Task

### Objectives

* Predict the department-wide sales for each store for the following year
* Model the effects of markdowns on holiday weeks
* Provide recommended actions based on the insights drawn, with prioritization placed on largest business impact

# Acquire

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings


In [2]:
sales = pd.read_csv("sales data-set.csv")

In [3]:
store = pd.read_csv("stores data-set.csv")

In [4]:
features = pd.read_csv("features data set.csv")

In [5]:
sales.shape

(421570, 5)

# Prepare

In [6]:
# Need to turn Date into a datetime df

sales.Date = pd.to_datetime(sales.Date)
features.Date = pd.to_datetime(features.Date)

In [7]:
# Are there any null values?

sales.isnull().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

In [8]:
store.isnull().sum()

Store    0
Type     0
Size     0
dtype: int64

In [9]:
features.isnull().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [10]:
features.Date

0      2010-05-02
1      2010-12-02
2      2010-02-19
3      2010-02-26
4      2010-05-03
          ...    
8185   2013-06-28
8186   2013-05-07
8187   2013-12-07
8188   2013-07-19
8189   2013-07-26
Name: Date, Length: 8190, dtype: datetime64[ns]

It seens that the only df with null values is `features`. The features missing are mostly the markdowns (which we expected, as they began recording them starting in 2011. We also seem to be missing about 500 datapoints for `CPI` and `Unemployment`. I am still not sure how I want to use these features specifically, but I am likely going to replace them with zero, or another close value.

In [11]:
# Lets merge the df

df = sales.merge(store, how="left", on="Store")
df = df.merge(features, how="left", on=["Date", "Store"])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421570 entries, 0 to 421569
Data columns (total 17 columns):
Store           421570 non-null int64
Dept            421570 non-null int64
Date            421570 non-null datetime64[ns]
Weekly_Sales    421570 non-null float64
IsHoliday_x     421570 non-null bool
Type            421570 non-null object
Size            421570 non-null int64
Temperature     421570 non-null float64
Fuel_Price      421570 non-null float64
MarkDown1       150681 non-null float64
MarkDown2       111248 non-null float64
MarkDown3       137091 non-null float64
MarkDown4       134967 non-null float64
MarkDown5       151432 non-null float64
CPI             421570 non-null float64
Unemployment    421570 non-null float64
IsHoliday_y     421570 non-null bool
dtypes: bool(2), datetime64[ns](1), float64(10), int64(3), object(1)
memory usage: 52.3+ MB


In [13]:
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-05-02,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,2010-12-02,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,1,2010-05-03,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [14]:
df = df.set_index("Date")

df = df.sort_index()

In [15]:
df.index.min()

Timestamp('2010-01-10 00:00:00')

In [16]:
# How do we handle the missing values?

df.loc["2011-11-01":"2011-11-12"]

,Store,Dept,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
Date,,,,,,,,,,,,,,,,
2011-11-02,29,35,1917.03,True,B,93638,28.89,3.255,NaN,NaN,NaN,NaN,NaN,133.260871,10.256,True
2011-11-02,30,6,7.44,True,C,42988,34.61,3.022,NaN,NaN,NaN,NaN,NaN,212.592862,8.028,True
2011-11-02,23,34,20330.29,True,B,114533,21.52,3.255,NaN,NaN,NaN,NaN,NaN,133.260871,5.114,True
2011-11-02,9,6,2410.44,True,B,125833,34.13,3.022,NaN,NaN,NaN,NaN,NaN,216.496729,6.416,True
2011-11-02,36,87,269.96,True,A,39910,41.16,3.009,NaN,NaN,NaN,NaN,NaN,211.698509,8.395,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-11-11,37,81,10617.52,False,C,39910,63.50,3.297,8.62,56.53,117.92,NaN,796.28,216.721737,7.716,False
2011-11-11,33,14,900.94,False,A,39690,58.75,3.677,105.63,126.28,5.06,NaN,27754.23,129.816710,8.010,False
2011-11-11,32,82,20501.28,False,A,203007,38.10,3.505,21104.84,7842.47,47.56,3908.03,4246.86,195.637694,8.513,False
